In [1021]:
import torch
import pomdp
%load_ext autoreload
%autoreload 2
%matplotlib inline

dtype = torch.float64

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1035]:
T, O, R = pomdp.env_simple(incorrect=0, coherence=1)
simple_pomdp_policy = pomdp.env_simple_fsc()
# some baselines... see what learning does?
spa, sps = pomdp.fsc_to_stochastic(simple_pomdp_policy[0], simple_pomdp_policy[1], T, dtype=dtype)
print(pomdp.policy_evaluation_sr(spa, sps, T, O, R, 0.95, dtype=dtype))
spa, sps = pomdp.random_fsc(2, T, O, R, dtype=dtype)
pomdp.policy_evaluation_sr(spa, sps, T, O, R, 0.95)

tensor([[20.0000, 19.0000],
        [19.0000, 20.0000]], dtype=torch.float64)


tensor([[ 9.8684, 10.1887],
        [ 9.8967, 10.1614]], dtype=torch.float64)

In [1040]:
np.random.seed(42)
T, O, R = pomdp.tiger(dtype=dtype)
fsc_action, fsc_state = pomdp.random_fsc(4, T, O, R, action_condition=False, dtype=dtype)
print(pomdp.policy_evaluation_stoch_old(fsc_action, fsc_state, T, O, R, 0.95, vi_eps=1e-10, dtype=dtype))
print(pomdp.policy_evaluation_stoch(fsc_action, fsc_state, T, O, R, 0.95, vi_eps=1e-10, dtype=dtype))
print(pomdp.policy_evaluation_sr(fsc_action, fsc_state, T, O, R, 0.95, dtype=dtype))
'''
exact, for action_condition=True
tensor([[-597.8844, -575.5487],
        [-580.2245, -596.7403],
        [-601.3115, -571.2379],
        [-571.1779, -589.2743]])

exact, for action_condition=False
tensor([[-604.2139, -582.0386],
        [-587.4158, -603.8062],
        [-607.6362, -578.1958],
        [-578.3529, -596.7704]])
'''
None

Converged 328
tensor([[-604.2143, -582.0390],
        [-587.4163, -603.8065],
        [-607.6367, -578.1964],
        [-578.3533, -596.7708]], dtype=torch.float64)
Converged 353
tensor([[-604.2143, -582.0390],
        [-587.4163, -603.8065],
        [-607.6367, -578.1964],
        [-578.3533, -596.7708]], dtype=torch.float64)
tensor([[-604.2143, -582.0390],
        [-587.4163, -603.8065],
        [-607.6367, -578.1964],
        [-578.3533, -596.7708]], dtype=torch.float64)


# Solving a simple one

In [1042]:
# grad works...
env = pomdp.env_simple(incorrect=0, coherence=1) # very simple
env = pomdp.env_simple()
d = pomdp.policy_iteration_grad(env, 0.95, steps=200, progress=25)
log_fsc(d['fsc_action'], d['fsc_state'])

24 -14.019098312194117
49 -15.044602820375507
74 -15.105074524070643
99 -15.126775652355265
124 -15.140861736378728
149 -15.151133326293824
174 -15.158883098814293
199 -15.164868833800828
action, obs, node
node 0 action p(a=0) 0.999
p(n=0|o=0) 1.0
p(n=1|o=1) 0.999
node 1 action p(a=1) 0.999
p(n=0|o=0) 0.999
p(n=1|o=1) 1.0


In [1047]:
d=pomdp.policy_iteration_bpi(pomdp.env_simple(incorrect=-1, coherence=0.9), 0.95, seed=805586775)
log_fsc(d['fsc_action'], d['fsc_state'])

Seed 805586775
0 1.437
1 8.444
2 15.2
3 15.2
converged 3
4 15.2
converged 4
5 15.2
converged 5
6 15.2
converged 6
7 15.2
converged 7
8 15.2
converged 8
9 15.2
converged 9
action, obs, node
node 0 action p(a=1) 1.0
p(n=0|a=0,o=0) 0.341
p(n=1|a=0,o=0) 0.659
p(n=0|a=0,o=1) 0.659
p(n=1|a=0,o=1) 0.341
p(n=1|a=1,o=0) 1.0
p(n=0|a=1,o=1) 1.0
node 1 action p(a=0) 1.0
p(n=1|a=0,o=0) 1.0
p(n=0|a=0,o=1) 1.0
p(n=0|a=1,o=0) 0.164
p(n=1|a=1,o=0) 0.836
p(n=0|a=1,o=1) 0.17
p(n=1|a=1,o=1) 0.83


# Tiger!

In [1048]:
pomdp.policy_evaluation_sr(*(pomdp.fsc_to_stochastic(*(*pomdp.optimal_tiger_fsc(), pomdp.tiger()[0]))+pomdp.tiger()+(0.95,)))

tensor([[ 19.3714,  19.3714],
        [ 24.6957,   3.0148],
        [ 28.4028, -81.5972],
        [  3.0148,  24.6957],
        [-81.5972,  28.4028]], dtype=torch.float64)

In [1049]:
# Tiger doesn't work...
d=policy_iteration_bpi(pomdp.tiger(), 0.995, num_nodes=5, debug=True, steps=2)
log_fsc(d['fsc_action'], d['fsc_state'])

Seed 940253173
iter: 0, pri_resid: 5.66902e+02, dual_resid: 1.16022e+06, mu: 2.29234e+02
iter: 1, pri_resid: 6.61559e+04, dual_resid: 1.13028e-10, mu: 3.03743e+00
iter: 2, pri_resid: 6.68278e+01, dual_resid: 1.72143e-12, mu: 5.30481e-02
iter: 3, pri_resid: 3.40992e+00, dual_resid: 1.95658e-12, mu: 2.85144e-03
iter: 4, pri_resid: 3.33633e-03, dual_resid: 2.50042e-12, mu: 3.05756e-06
iter: 5, pri_resid: 7.78460e-06, dual_resid: 1.99444e-12, mu: 3.05756e-09
iter: 6, pri_resid: 1.10578e-08, dual_resid: 1.85308e-12, mu: 3.05756e-12
iter: 7, pri_resid: 1.27660e-11, dual_resid: 1.40938e-12, mu: 3.05756e-15
iter: 8, pri_resid: 9.09495e-13, dual_resid: 1.53608e-12, mu: 3.05756e-18
iter: 9, pri_resid: 5.56515e-17, dual_resid: 1.66527e-12, mu: 3.05756e-21
iter: 10, pri_resid: nan, dual_resid: nan, mu: nan
iter: 11, pri_resid: nan, dual_resid: nan, mu: nan
iter: 12, pri_resid: nan, dual_resid: nan, mu: nan
iter 0 node 0 before tensor([[-5909.0712, -5887.1001],
        [-5919.2223, -5890.7275],
   

# optimize POMDP params

This is a trivial example of tuning a POMDP's parameters to optimize some objective. Here, I optimize the coherence of observations w.r.t. task utility. The optimal solutions either have 0% or 100% coherence.

In [1098]:
def optimize_pomdp(*, start=None, gamma=0.95, lr=1e-3, steps=100, progress=10, dtype=torch.float64, num_nodes=2, bpi=True, seed=None):
    def value(coherence):
        env = pomdp.env_simple(coherence=coherence)
        kw = dict(progress=False, dtype=dtype, num_nodes=num_nodes)
        if bpi:
            r = pomdp.policy_iteration_bpi(env, gamma, steps=40, debug=False, **kw)
        else:
            r = pomdp.policy_iteration_grad(env, gamma, steps=100, **kw)
        return r['value']

    start = np.random.uniform(-0.4, +0.4, size=1) if start is None else np.array([float(start)])
    coherence_logit = torch.tensor(start).requires_grad_(True).type(dtype)
    opt = torch.optim.SGD([coherence_logit], lr=lr)

    for idx in range(steps):
        opt.zero_grad()

        loss = -value(coherence_logit.sigmoid())
        loss.backward(retain_graph=True)

        if ((idx+1) % progress) == 0:
            print(idx, 'value', round(-loss.item(), 3), 'coherence', round(coherence_logit.sigmoid().detach().item(), 3))

        opt.step()

print('SGD-POMDP')
optimize_pomdp(start=0.5, progress=1, steps=10, bpi=False)
print('BPI-POMDP')
optimize_pomdp(start=0.1, progress=1, steps=2, lr=1e-12, bpi=True)

SGD-POMDP
0 value 4.631 coherence 0.622
1 value 10.812 coherence 0.786
2 value 13.788 coherence 0.865
3 value 15.332 coherence 0.905
4 value 16.177 coherence 0.928
5 value 16.685 coherence 0.941
6 value 17.054 coherence 0.951
7 value 17.315 coherence 0.958
8 value 17.512 coherence 0.963
9 value 17.653 coherence 0.967
BPI-POMDP
0 value 0.172 coherence 0.525
1 value 19.0 coherence 1.0


# old

In [849]:
import pomdp
T, O, R = pomdp.env_simple(incorrect=0, coherence=1)

num_states, num_actions, _ = T.shape
num_obs = O.shape[2]

# canz -> p(n' | n, a, o) -> p(n', a, o | n) ??
# c_a -> p(a | n)
num_nodes = 5
fsc_action = torch.ones((num_nodes, num_actions))/num_actions
#fsc_state = torch.ones((num_nodes, num_actions, num_obs, num_nodes))/(num_actions+num_obs+num_nodes)
#fsc_state = torch.ones((num_nodes, num_obs, num_nodes))/(num_nodes)
fsc_state = torch.ones((num_nodes, num_actions, num_obs, num_nodes))/(num_actions+num_obs+num_nodes)

num_nodes = 2
np.random.seed(41)
fsc_action, fsc_state = pomdp.random_fsc(num_nodes, T, O, R, dtype=dtype, action_condition=True)
#fsc_action, fsc_state = det_to_stoch(opt_fsc_action, opt_fsc_state, T)

#fsc_action = torch.tensor(np.random.uniform(0, 1, size=(num_nodes, num_actions))).softmax(1).float()
#xrnd = np.random.uniform(0, 1, size=(num_nodes, num_actions, num_obs, num_nodes))
#fsc_state = torch.tensor(xrnd/np.sum(xrnd, axis=(1, 2, 3))).float()

canz_shape = torch.Size((num_actions, num_obs, num_nodes))

debug=True

gamma = 0.95
dtype = torch.float64
steps = 4

V = pomdp.policy_evaluation_sr(fsc_action, fsc_state, T, O, R, gamma, dtype=dtype)
#V = pomdp.policy_evaluation_stoch(fsc_action, fsc_state, T, O, R, debug=False)
for i in range(steps):
    prev = V.clone()
    print(i)
    for n in range(num_nodes):
        V = pomdp.policy_evaluation_sr(fsc_action, fsc_state, T, O, R, gamma, dtype=dtype)
        if debug:
            print('value', V)
        #V = policy_evaluation_stoch(fsc_action, fsc_state, T, O, R, debug=False)
        constraints, unpack = pomdp.make_constraints_simple(n, V, T, O, R, gamma, dtype=dtype, qcoef=1e-10)
        #soln = qpth.qp.QPFunction(verbose=True, solver=qpth.qp.QPSolvers.CVXPY)(*constraints).float()
        soln = qpth.qp.QPFunction(verbose=debug)(*constraints).float()
        #delta = soln[0, 0].item()
        delta, c_a, canz = unpack(soln[0])
        print('node', n, 'V delta', delta)
        if debug:
            print('node', n, 'V delta', delta)
            print('sums', c_a.sum(), canz.sum(), 'c_a', c_a)
        if True:#delta > 0: # HACK check tos ee if xxx
            #fsc_action[n] = soln[0, c_a_slice]
            #fsc_action[n] = c_a
            # normalize!!??
            #fsc_action[n] = fsc_action[n] / fsc_action[n].sum()
            #do we marginalize over actions?
            # do we normalize so this is P(n'|n,o,a), instead of p(a,n'|n,o)?
            #canz = soln[0, canz_slice].view(canz_shape)
            # normalizing by p(a|n) to give us p(n'|n,o)
            #fsc_state[n] = canz/canz.sum(0)
            # HACK actually let's marginalize over actions first then normalize
            # HACK actually we need to change this per the later meuleau
            #canz = canz.sum(0)
            #print(canz, canz.shape, canz.sum(0).shape, fsc_state[n].shape)
            #print(canz, canz.shape, canz.sum(1), (canz/canz.sum(1)[None, :]).sum(1), c_a)
            #fsc_state[n] = canz/canz.sum(1)[:, None]

            # NEW
            fsc_action[n] = c_a
            fsc_state[n] = canz/canz.sum(-1)[:, :, None]

    V = pomdp.policy_evaluation_sr(fsc_action, fsc_state, T, O, R, gamma, dtype=dtype)
    print(i, V, (prev-V).norm())
    if (prev-V).norm() < 1e-3:
        print('converged!')
        #break


0
value tensor([[10.3875,  9.6378],
        [10.4890,  9.5473]], dtype=torch.float64)
iter: 0, pri_resid: 4.82224e+00, dual_resid: 3.86640e+01, mu: 1.74219e+00
iter: 1, pri_resid: 2.35487e-03, dual_resid: 4.58637e-15, mu: 1.41030e-01
iter: 2, pri_resid: 9.74553e-05, dual_resid: 2.31790e-15, mu: 7.61143e-03
iter: 3, pri_resid: 1.42890e-05, dual_resid: 1.92618e-15, mu: 3.51400e-03
iter: 4, pri_resid: 1.23231e-06, dual_resid: 2.34121e-15, mu: 4.32708e-05
iter: 5, pri_resid: 1.89140e-09, dual_resid: 1.49275e-15, mu: 4.34379e-08
iter: 6, pri_resid: 1.88965e-11, dual_resid: 1.83779e-15, mu: 4.34379e-11
iter: 7, pri_resid: 2.04913e-14, dual_resid: 1.58835e-15, mu: 4.34379e-14
node 0 V delta -4.694807529449463
node 0 V delta -4.694807529449463
sums tensor(1.) tensor(1.) c_a tensor([2.3156e-12, 1.0000e+00])
value tensor([[12.6476, 15.4626],
        [13.0745, 14.4330]], dtype=torch.float64)
iter: 0, pri_resid: 4.89006e+00, dual_resid: 6.07113e+01, mu: 2.08440e+00
iter: 1, pri_resid: 1.76566e-03,